In [1]:
import tensorflow as tf
import keras
import time

2025-07-03 04:01:17.285168: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 04:01:17.342385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 04:01:17.342450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 04:01:17.344972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 04:01:17.355731: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.applications.ConvNeXtLarge(include_top=False, input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy'])
    
    return model


def get_dataset():
    batch_size_per_replica = 512
    batch_size = batch_size_per_replica * strategy.num_replicas_in_sync
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

class timecallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.times = []
        # use this value as reference to calculate cummulative time taken
        self.start_time = time.perf_counter()
        self.end_time = time.perf_counter()
    def on_epoch_begin(self,epoch,logs = {}):
        self.start_time = time.perf_counter()
    def on_epoch_end(self,epoch,logs = {}):
        self.end_time = time.perf_counter()
        print('\nepoch time: ',self.end_time - self.start_time)

# Using 4 GPUs

In [3]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"]) # use 4 GPUs
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2025-07-03 04:01:20.301886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43358 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:03:00.0, compute capability: 8.9
2025-07-03 04:01:20.303099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43358 MB memory:  -> device: 1, name: NVIDIA L40S, pci bus id: 0000:04:00.0, compute capability: 8.9
2025-07-03 04:01:20.304386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43358 MB memory:  -> device: 2, name: NVIDIA L40S, pci bus id: 0000:63:00.0, compute capability: 8.9
2025-07-03 04:01:20.305231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 43358 MB memory:  -> device: 3, name: NVIDIA L40S, pci bus id: 0000:64:00.0, compute capability: 

Number of devices: 4


In [4]:
%%time      
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()
        
# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
timetaken = timecallback()
model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks = [timetaken])

# Test the model on all available devices.
model.evaluate(test_dataset)

Epoch 1/50
INFO:tensorflow:Collective all_reduce tensors: 346 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Collective all_reduce tensors: 346 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost

2025-07-03 04:04:01.388218: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-07-03 04:04:01.460863: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-07-03 04:04:01.481777: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-07-03 04:04:01.499874: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90100
2025-07-03 04:04:02.142048: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc72e6c4d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-03 04:04:02.142096: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2025-07-03 04:04:02.142106: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA L40S, Compute Capability 8.9
2025-07-03 04:04:02.142118: I external/local_xla/xla

20/20 [==============================] - ETA: 0s - loss: 2.6210 - accuracy: 0.0973INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).

epoch time:  229.61116927303374
20/20 [==============================] - 230s 2s/step - loss: 2.6210 - accuracy: 0.0973 - val_loss: 2.3026 - val_accuracy: 0.1015
Epoch 2/50
20/20 [==============================] - ETA: 0s - loss: 2.3026 - accuracy: 0.0995
epoch time:  7.9956765323877335
20/20 [==============================] - 8s 387ms/step - loss: 2.3026 - accuracy: 0.0995 - val_loss: 2.3026 - val_accuracy: 0.1015
Epoch 3/50
20/20 [==============================] - ETA: 0s - loss: 2.3026 - accuracy: 0.0995
epoch time:  7.724391974508762
20/20 [==============================] - 8s 388ms/step - loss: 2.3026 - accuracy: 0.0995 - va

[2.302597761154175, 0.1000814139842987]